In [3]:
import requests



In [ ]:
def get_queue_amount():
    # Make a GET request to the specified URL
    response = requests.get("http://127.0.0.1:8188/queue")

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        # Calculate the sum of "queue_running" and "queue_pending"
        total = len(data["queue_running"]) + len(data["queue_pending"])
        return total
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")
        return -1


In [ ]:
print(get_queue_amount())

4
